# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:/Users/kgogi/DSI/scaling_to_production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [5]:
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

#X.head(), Y.head()

import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)* 
#The "random state" in data splitting functions controls the randomness of how the data is divided. It's useful because:

Reproducibility: It ensures you get the same split every time you run the code, making your results consistent.
Comparability: It allows others to replicate your work and compare results under the same conditions.
Control: It helps you manage randomness, making it easier to isolate the effects of other variables in your experiments.


# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [ ]:
X

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB

num_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

pip_num_KNN = Pipeline([
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
   ('robust_scaler', RobustScaler())
])

pip_cat_simple = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

ctransform = ColumnTransformer(
    transformers=[
        ('num_transforms', pip_num_KNN, num_cols),
        ('cat_transforms', pip_cat_simple, cat_cols)
    ]
)

ctransform

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [7]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('preprocessing', ctransform),
    ('classifier', RandomForestClassifier())
])

pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robust_scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [8]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']
res_dic = cross_validate(pipeline, X_train, Y_train, cv = 5, scoring = scoring, return_train_score=True)

c:\Users\kgogi\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\kgogi\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [9]:
res_dic_df = pd.DataFrame(res_dic).sort_values(by='test_neg_log_loss')
print(res_dic_df)

   fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
2  9.008967    0.258231          -0.386801           -0.081135      0.902350   
4  8.823868    0.210445          -0.372550           -0.081139      0.902723   
1  8.991919    0.205040          -0.366080           -0.081372      0.902595   
0  8.618102    0.186985          -0.343372           -0.081461      0.904887   
3  9.843291    0.177506          -0.342794           -0.081788      0.907435   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
2            1.0       0.852567        0.999945                0.773157   
4            1.0       0.856077        0.999945                0.774243   
1            1.0       0.847774        0.999945                0.768607   
0            1.0       0.851064        1.000000                0.776310   
3            1.0       0.859149        1.000000                0.783655   

   train_balanced_accuracy  
2                 0.999887  
4         

Calculate the mean of each metric. 

In [10]:
res_dic_mean = res_dic_df.mean()
print(res_dic_mean)

fit_time                   9.057230
score_time                 0.207641
test_neg_log_loss         -0.362319
train_neg_log_loss        -0.081379
test_roc_auc               0.903998
train_roc_auc              1.000000
test_accuracy              0.853326
train_accuracy             0.999967
test_balanced_accuracy     0.775195
train_balanced_accuracy    0.999932
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [11]:
pipeline.fit(X_train, Y_train)
Y_pred_test = pipeline.predict(X_test)
Y_proba_test = pipeline.predict_proba(X_test)

performance_metrics = {
    'neg_log_loss': log_loss(Y_test, Y_proba_test),
    'roc_auc': roc_auc_score(Y_test, Y_proba_test[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred_test),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred_test)

}

print(performance_metrics)

{'neg_log_loss': 0.3800908172688724, 'roc_auc': 0.9008020175214353, 'accuracy': 0.8532091309243526, 'balanced_accuracy': 0.7712067275447557}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.) 
Recoding the target variable income immediately after loading the data is convenient because it simplifies the data preparation process. It ensures that the target variable is in the desired format (numeric) right from the start, making subsequent analysis and modeling steps easier and more consistent.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.